In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
impressions_df = pd.read_csv("../datasets/impressions/impressions_groupm_tele2_6_months.csv")

In [3]:
client_df = pd.read_excel("../datasets/groupm_tele2_report.xlsx")
client_df

,Date,Hour,LI Id,LI Name,Placement Id,Size,domain,Imps,Viewability Measurement Rate %,Viewability Rate %,Custom Viewability Rate %
0,2023-02-01,13,20212601,Xaxis Premium 50 - WL - Attention Algo - AS10 ...,19499490,320x320,aftonbladet.se,943,0.958643,0.632743,0.440265
1,2023-02-01,13,20212601,Xaxis Premium 50 - WL - Attention Algo - AS10 ...,19499420,300x600,aftonbladet.se,907,0.983462,0.569507,0.215247
2,2023-02-01,13,20212601,Xaxis Premium 50 - WL - Attention Algo - AS10 ...,19499395,320x320,itunes.apple.com/us/app/sportbladet/id429681201,900,0.994444,0.858101,0.776536
3,2023-02-01,13,20212601,Xaxis Premium 50 - WL - Attention Algo - AS10 ...,19499428,640x320,play.google.com/store/apps/details?id=se.afton...,869,0.989643,0.538372,0.319767
4,2023-02-01,13,20212601,Xaxis Premium 50 - WL - Attention Algo - AS10 ...,20148638,640x320,expressen.se,777,0.994852,0.723157,0.476067
...,...,...,...,...,...,...,...,...,...,...,...
278105,2023-04-30,23,20178378,Xaxis Premium 50 - IAS Contextual & Grapeshot ...,20472416,320x320,unt.se,1,1.000000,0.000000,0.000000
278106,2023-04-30,23,20178378,Xaxis Premium 50 - IAS Contextual & Grapeshot ...,20616291,320x320,expressen.se,1,1.000000,0.000000,0.000000
278107,2023-04-30,23,20178380,Cookieless Xaxis Premium 50 - IAS Contextual ...,16346970,320x480,abcnyheter.no,1,1.000000,0.000000,0.000000
278108,2023-04-30,23,20178380,Cookieless Xaxis Premium 50 - IAS Contextual ...,20472359,320x320,mvt.se,1,1.000000,0.000000,0.000000


In [4]:
import json

def extract_pid(placement_id):
    placement_id = json.loads(placement_id)
    bam_ad_slots = placement_id["bam_ad_slot"]
    final_bam_ad_slots = [bas for bas in bam_ad_slots if bas.strip() != '']
    try:
        ret = placement_id['tag_id'][0]
    except:
        return
    ret += ','.join([bas for bas in final_bam_ad_slots])
    return ret

def pid_type(pid):
    try:
        int(pid)
        return 'int'
    except:
        return 'str'

impressions_df['pid'] = impressions_df['placement_ids'].apply(extract_pid)
impressions_df['pid_type'] = impressions_df['pid'].apply(pid_type)

In [5]:
# run through predictions
import os
import json


def _get_pred_map():
    pred_map = dict()
    for _i in range(1,2):
        if _i == 1:
            _year = 2022
            _range1 = 11
            _range2 = 13
        else:
            _year = 2023
            _range1 = 1
            _range2 = 5
        for i in range(_range1, _range2):
            if i < 10:
                i = f'0{i}'
            for day in os.listdir(f'../predictions/{_year}/{i}'):
                for h in os.listdir(f'../predictions/{_year}/{i}/{day}'):
                    if not h.endswith('.ndjson'):
                        continue
                    with open(f'../predictions/{_year}/{i}/{day}/{h}')as f:
                        for line in f.readlines():
                            json_line = json.loads(line)
                            id_ = json_line['id']
                            pred_map[id_] = json_line['prediction']
    return pred_map
                            
pred_map = _get_pred_map()

In [6]:
impressions_df = impressions_df[impressions_df["pid_type"] == 'int']
impressions_df['pid'] = impressions_df['pid'].apply(lambda pid: int(pid))
impressions_df

,id,hostname,beacon_ids,ad_width_chosen,ad_height_chosen,placement_ids,placement_ids_chosen,total_fixation_duration,channel,beacon_type_chosen,ad_technical_format,is_fixated,gaze_valid,is_iab_inview,exist_viewable_1_s_threshold_50,exist_viewable_2_s_threshold_50,chosen_brand,pid,pid_type
71,884d5666-860e-43eb-ac7c-971b71de43d0,wrestlinginc.com,"[""3ab197b3-12e9-4608-b684-6c99b847aa91""]",180,150,"{""tag_id"":[""25909932""],""bam_ad_slot"":["""","""",""""]}",NaN,0,display,ad/anxv,display,False,False,False,False,False,NaN,25909932,int
91,ac246180-4731-42be-9665-350bc1a68f24,wrestlinginc.com,"[""6213e67e-2806-42e5-a733-1accc9eb1366""]",250,480,"{""tag_id"":[""17112722""],""bam_ad_slot"":["""","""",""""]}",NaN,0,display,ad/adnxs.com-a-img-v1,display,False,False,True,True,True,NaN,17112722,int
128,c6be85eb-4711-49f1-b215-0336d1c68001,wrestlinginc.com,"[""ea1067b5-9bec-422f-8a8a-a084a5323fe8""]",180,150,"{""tag_id"":[""28010828""],""bam_ad_slot"":["""","""",""""]}",NaN,0,display,ad/anxv,display,False,True,False,False,False,NaN,28010828,int
158,8785c26e-c16d-44cc-8aa1-1d6de70ac8fa,blocket.se,"[""5e7ce408-4e82-42be-9982-2ffb6f2571f0"",""e30f6...",980,240,"{""tag_id"":[""19499532""],""bam_ad_slot"":["""","""",""""]}",NaN,0,display,ad/anxv,display,False,False,True,True,True,NaN,19499532,int
159,ff2c49d5-d31f-433e-847d-dc02bc15ace5,blocket.se,"[""2df3b120-1ba5-4a09-8b54-f56d644a746e"",""7a9b2...",160,600,"{""tag_id"":[""19499533""],""bam_ad_slot"":["""","""",""""]}",NaN,0,display,ad/adnxs.com-a-img-v1,display,False,False,True,True,True,NaN,19499533,int
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9066786,3913b6c2-7530-4398-875e-dec9cce5c6a6,blocket.se,"[""1cb8a8db-e0af-4a55-a95a-834b22c3ddf1"",""2b369...",1180,240,"{""tag_id"":[""19499515""],""bam_ad_slot"":["""","""",""""]}",NaN,0,display,ad/tagid-iframe-v1,display,False,False,True,True,True,NaN,19499515,int
9066792,cc31a68b-6772-46ef-9d94-93ab88821b49,svd.se,"[""1aa68f34-6079-4e91-b20a-70f8df1defa7"",""5c396...",1180,240,"{""tag_id"":[""19499705""],""bam_ad_slot"":["""","""",""""]}",NaN,17796,display,ad/newssites-banners-v1,display,True,True,True,True,True,NaN,19499705,int
9066799,25e65557-1cc8-47de-9add-a2e379076ce0,svd.se,"[""16291369-110d-4c67-a662-04312afd7476"",""69a77...",980,360,"{""tag_id"":[""19499712""],""bam_ad_slot"":["""","""",""""]}",NaN,163648,display,ad/newssites-banners-v1,display,True,True,True,True,True,NaN,19499712,int
9066800,cc09d11a-10c6-4932-94a9-fad75f1b76c0,svd.se,"[""4572f4b2-e564-4746-8d67-54f86bca919c"",""d1718...",1180,240,"{""tag_id"":[""19499705""],""bam_ad_slot"":["""","""",""""]}",NaN,604,display,ad/newssites-banners-v1,display,True,True,True,True,True,NaN,19499705,int


In [7]:
tele2_placements = list(client_df['Placement Id'].unique())
tele2_impressions = impressions_df[impressions_df['pid'].isin(tele2_placements)]
tele2_impressions

,id,hostname,beacon_ids,ad_width_chosen,ad_height_chosen,placement_ids,placement_ids_chosen,total_fixation_duration,channel,beacon_type_chosen,ad_technical_format,is_fixated,gaze_valid,is_iab_inview,exist_viewable_1_s_threshold_50,exist_viewable_2_s_threshold_50,chosen_brand,pid,pid_type
158,8785c26e-c16d-44cc-8aa1-1d6de70ac8fa,blocket.se,"[""5e7ce408-4e82-42be-9982-2ffb6f2571f0"",""e30f6...",980,240,"{""tag_id"":[""19499532""],""bam_ad_slot"":["""","""",""""]}",NaN,0,display,ad/anxv,display,False,False,True,True,True,NaN,19499532,int
159,ff2c49d5-d31f-433e-847d-dc02bc15ace5,blocket.se,"[""2df3b120-1ba5-4a09-8b54-f56d644a746e"",""7a9b2...",160,600,"{""tag_id"":[""19499533""],""bam_ad_slot"":["""","""",""""]}",NaN,0,display,ad/adnxs.com-a-img-v1,display,False,False,True,True,True,NaN,19499533,int
160,afb9d8df-bedc-4e5a-8e4e-f05e3953d8e6,blocket.se,"[""349b8b43-cae3-4c9b-8372-e9a621b2529a"",""7a9b2...",160,600,"{""tag_id"":[""19499533""],""bam_ad_slot"":["""","""",""""]}",NaN,0,display,ad/adnxs.com-iframe-v3,display,False,False,True,True,True,NaN,19499533,int
164,df108124-f840-4f2f-b26a-8ce8723f0ba1,blocket.se,"[""7385358c-0e4d-4f83-a6c6-5ef02b0b1422"",""f49d4...",980,240,"{""tag_id"":[""19499532""],""bam_ad_slot"":["""","""",""""]}",NaN,0,display,ad/anxv,display,False,False,False,False,False,NaN,19499532,int
165,a62cea16-e097-4504-bfa6-1cb38d6a937d,blocket.se,"[""ab203856-01d7-491e-82ca-216350bafbf9"",""f49d4...",980,240,"{""tag_id"":[""19499532""],""bam_ad_slot"":["""","""",""""]}",NaN,0,display,ad/adnxs.com-iframe-v3,display,False,False,False,False,False,NaN,19499532,int
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9066580,ac42b9bb-8e76-447e-98e8-85688cff3f51,aftonbladet.se,"[""1ec07a50-a35e-4bfb-abdc-b6cd87419347"",""5afe0...",468,240,"{""tag_id"":[""19499410""],""bam_ad_slot"":["""","""",""""]}",NaN,695,out-stream,ad/doubleclick.net-ins-dcmads-v1,out-stream,True,True,True,True,True,NaN,19499410,int
9066630,37eb283f-5986-4ca5-b8c0-b8d31ab78a8a,aftonbladet.se,"[""2e95ea7e-5940-4b5f-a7f1-97f32ef5dbca"",""4e442...",250,480,"{""tag_id"":[""19499420""],""bam_ad_slot"":["""","""",""""]}",NaN,30,display,ad/tagid-iframe-v1,display,False,True,False,False,False,NaN,19499420,int
9066763,875f4018-1e77-4667-b3ca-6a5fcd93d621,aftonbladet.se,"[""dc2ea2e8-6c4e-4043-b685-c0a78509701f"",""e46ff...",980,240,"{""tag_id"":[""19499441""],""bam_ad_slot"":["""","""",""""]}",NaN,272,display,ad/tagid-iframe-v1,display,True,True,True,True,True,NaN,19499441,int
9066765,57e484c9-85d8-4fb8-84e6-7ce27b90698c,aftonbladet.se,"[""b562b466-1962-4193-96cd-8331f2b7fdf5""]",468,240,"{""tag_id"":[""19499428""],""bam_ad_slot"":["""","""",""""]}",NaN,0,display,ad/adnxs.com-iframe-v3,display,False,False,True,True,True,NaN,19499428,int


In [8]:
def _get_final_fixation(gaze_valid, is_fixated, id_, pred_map):
    if gaze_valid:
        return is_fixated
    if id_ in pred_map:
        return pred_map[id_]
    return False

tele2_impressions['final_fixation'] = tele2_impressions.apply(lambda row: _get_final_fixation(row['gaze_valid'], row['is_fixated'], row['id'], pred_map), axis=1)

/tmp/ipykernel_22324/271553796.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tele2_impressions['final_fixation'] = tele2_impressions.apply(lambda row: _get_final_fixation(row['gaze_valid'], row['is_fixated'], row['id'], pred_map), axis=1)


In [9]:
tele2_impressions["true_is_iab_inview"] = tele2_impressions.apply(
    lambda x: 
    True if (
        (x["ad_technical_format"] == "out-stream") & (x["exist_viewable_1_s_threshold_50"] == True) & (x["exist_viewable_2_s_threshold_50"] == False)
    ) | (x["is_iab_inview"] == True) 
    else False, 
    axis=1
)

/tmp/ipykernel_22324/4002082058.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tele2_impressions["true_is_iab_inview"] = tele2_impressions.apply(


In [14]:
grouped_df = tele2_impressions.groupby(["pid"]).apply(
    lambda row: pd.Series(
        {
            "num_impressions": row["id"].count(),
            "num_fixations": row[row["final_fixation"] == True]["id"].count(),
            "num_inview": row.loc[row["true_is_iab_inview"], "id"].count(),
        }
    )
)
grouped_df = grouped_df.reset_index()
grouped_df["fix/imp_ratio"] = round(grouped_df["num_fixations"] / grouped_df["num_impressions"] * 100, 2)
grouped_df["inview/imp_ratio"] = round(grouped_df["num_inview"] / grouped_df["num_impressions"] * 100, 2)
grouped_df["fix/inview_ratio"] = round(grouped_df["num_fixations"] / grouped_df["num_inview"] * 100, 2)
grouped_df["fix/inview_ratio"] = grouped_df["fix/inview_ratio"].fillna(0)
grouped_df["fix/inview_ratio"] = grouped_df["fix/inview_ratio"].replace([np.inf, -np.inf], 0)
grouped_df

,pid,num_impressions,num_fixations,num_inview,fix/imp_ratio,inview/imp_ratio,fix/inview_ratio
0,6929499,13,0,0,0.00,0.00,0.00
1,6931042,86,27,16,31.40,18.60,168.75
2,6933343,3,0,0,0.00,0.00,0.00
3,6935443,2,0,0,0.00,0.00,0.00
4,6938322,1,0,0,0.00,0.00,0.00
...,...,...,...,...,...,...,...
266,24928807,13,1,8,7.69,61.54,12.50
267,25140105,1290,38,801,2.95,62.09,4.74
268,25659495,409,19,72,4.65,17.60,26.39
269,27264940,14,5,0,35.71,0.00,0.00


In [15]:
client_df_grouped = client_df.groupby(["domain", "Placement Id"]).apply(
    lambda x: pd.Series(
        {
            "num_imps": np.sum(x["Imps"])
        }
    )
)
client_df_grouped = client_df_grouped.reset_index()
client_df_grouped

,domain,Placement Id,num_imps
0,56kilo.se,20148891,645
1,56kilo.se,21462254,5
2,56kilo.se,21791822,33
3,90min.com,19760269,9
4,9gag.com,13703351,2734
...,...,...,...
1256,ystadsallehanda.se,19218493,1
1257,ystadsallehanda.se,19256766,43
1258,ystadsallehanda.se,20472257,18
1259,ystadsallehanda.se,20472258,27


In [20]:
merged_df = pd.merge(grouped_df, client_df_grouped, how='outer', left_on=['pid'], right_on=['Placement Id'])
merged_df

,pid,num_impressions,num_fixations,num_inview,fix/imp_ratio,inview/imp_ratio,fix/inview_ratio,domain,Placement Id,num_imps
0,6929499.0,13.0,0.0,0.0,0.0,0.0,0.00,hertson.se,6929499,21466
1,6931042.0,86.0,27.0,16.0,31.4,18.6,168.75,ifokus.se,6931042,875432
2,6933343.0,3.0,0.0,0.0,0.0,0.0,0.00,fixasjalv.se,6933343,1461
3,6935443.0,2.0,0.0,0.0,0.0,0.0,0.00,minboll.se,6935443,71812
4,6938322.0,1.0,0.0,0.0,0.0,0.0,0.00,recepten.se,6938322,12
...,...,...,...,...,...,...,...,...,...,...
1256,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ystadsallehanda.se,19218493,1
1257,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ystadsallehanda.se,19256766,43
1258,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ystadsallehanda.se,20472257,18
1259,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ystadsallehanda.se,20472258,27


In [21]:
def get_sample_size(num_impressions):
    if np.isnan(num_impressions):
        return "N/A"
    if num_impressions <= 20:
        return "low"
    elif num_impressions > 20 and num_impressions <= 100:
        return "medium"
    else:
        return "high"
    
def get_weight(x):
    if x == "high":
        return 1
    elif x == "medium":
        return 0.7
    elif x == "low":
        return 0.5
    else:
        return None
    
merged_df['sample_size'] = merged_df['num_impressions'].apply(lambda num_impressions: get_sample_size(num_impressions))
merged_df["weight"] = merged_df["sample_size"].apply(lambda x: get_weight(x))

merged_df

,pid,num_impressions,num_fixations,num_inview,fix/imp_ratio,inview/imp_ratio,fix/inview_ratio,domain,Placement Id,num_imps,sample_size,weight
0,6929499.0,13.0,0.0,0.0,0.0,0.0,0.00,hertson.se,6929499,21466,low,0.5
1,6931042.0,86.0,27.0,16.0,31.4,18.6,168.75,ifokus.se,6931042,875432,medium,0.7
2,6933343.0,3.0,0.0,0.0,0.0,0.0,0.00,fixasjalv.se,6933343,1461,low,0.5
3,6935443.0,2.0,0.0,0.0,0.0,0.0,0.00,minboll.se,6935443,71812,low,0.5
4,6938322.0,1.0,0.0,0.0,0.0,0.0,0.00,recepten.se,6938322,12,low,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...
1256,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ystadsallehanda.se,19218493,1,N/A,NaN
1257,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ystadsallehanda.se,19256766,43,N/A,NaN
1258,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ystadsallehanda.se,20472257,18,N/A,NaN
1259,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ystadsallehanda.se,20472258,27,N/A,NaN


In [33]:
merged_df = merged_df.dropna(subset=["num_imps"])
merged_df = merged_df.dropna(subset=["num_impressions"])
#merged_df["fix_ratio"] = round(merged_df["num_fixations"] / merged_df["num_impressions"], 2)
#merged_df["impression_inview_ratio"] = round(merged_df["num_inview"] / merged_df["num_imps"], 2)
#merged_df["client_fixations"] = merged_df.apply(lambda x: math.floor(x["fix/imp_ratio"] * x["num_imps"] * x["weight"]), axis=1)
#merged_df["client_fix/imp_ratio"] = round(merged_df["client_fixations"] / merged_df["num_imps"] * 100, 2)

#merged_df["client_inview/imp_ratio"] = merged_df["fix/inview_ratio"] * merged_df["client_fixations"]
#merged_df["client_inviews"] = round(merged_df["client_inview/imp_ratio"] * merged_df["num_imps"] * 100, 2)

#merged_df["client_fixation_inview_ratio"] = round(merged_df["client_fixations"] / merged_df["client_inviews"] * 100, 2)

merged_df["client_fixations"] = merged_df.apply(lambda x: math.floor((x["fix/imp_ratio"] / 100) * x["num_imps"] * x["weight"]), axis=1)
merged_df["client_inviews"] = merged_df.apply(lambda x: math.floor((x["inview/imp_ratio"] / 100) * x["num_imps"]), axis=1)
merged_df["client_fixation_inview_ratio"] = round(merged_df["client_fixations"] / merged_df["client_inviews"] * 100, 2)

# cleanup
merged_df["client_fixation_inview_ratio"] = merged_df["client_fixation_inview_ratio"].fillna(0)
merged_df["client_fixation_inview_ratio"] = merged_df["client_fixation_inview_ratio"].replace([np.inf, -np.inf], 0)

merged_df

,pid,num_impressions,num_fixations,num_inview,fix/imp_ratio,inview/imp_ratio,fix/inview_ratio,domain,Placement Id,num_imps,sample_size,weight,client_fixations,client_inviews,client_fixation_inview_ratio
0,6929499.0,13.0,0.0,0.0,0.00,0.00,0.00,hertson.se,6929499,21466,low,0.5,0,0,0.00
1,6931042.0,86.0,27.0,16.0,31.40,18.60,168.75,ifokus.se,6931042,875432,medium,0.7,192419,162830,118.17
2,6933343.0,3.0,0.0,0.0,0.00,0.00,0.00,fixasjalv.se,6933343,1461,low,0.5,0,0,0.00
3,6935443.0,2.0,0.0,0.0,0.00,0.00,0.00,minboll.se,6935443,71812,low,0.5,0,0,0.00
4,6938322.0,1.0,0.0,0.0,0.00,0.00,0.00,recepten.se,6938322,12,low,0.5,0,0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,24928807.0,13.0,1.0,8.0,7.69,61.54,12.50,vibilagare.se,24928807,91,low,0.5,3,56,5.36
267,25140105.0,1290.0,38.0,801.0,2.95,62.09,4.74,slashfilm.com,25140105,28818,high,1.0,850,17893,4.75
268,25659495.0,409.0,19.0,72.0,4.65,17.60,26.39,cyclingnews.com,25659495,114965,high,1.0,5345,20233,26.42
269,27264940.0,14.0,5.0,0.0,35.71,0.00,0.00,svd.se,27264940,18,low,0.5,3,0,0.00


In [39]:
grouped_df = merged_df.groupby(["domain", "pid"]).apply(
    lambda x: pd.Series(
        {
            "sum_imp": x["num_impressions"].sum(),
            "sum_imp_client": x["num_imps"].sum(),
            "sum_fixations": x["num_fixations"].sum(),
            "sum_fixations_client": x["client_fixations"].sum(),
            "sum_inview": x["num_inview"].sum(),
            "sum_inview_client": x["client_inviews"].sum(),
            "avg_weight": np.average(x["weight"]),
        }
    )
)
grouped_df = grouped_df.reset_index()

In [40]:
grouped_df["pid"] = grouped_df["pid"].astype("Int64")
grouped_df["sum_imp"] = grouped_df["sum_imp"].astype("Int64")
grouped_df["sum_imp_client"] = grouped_df["sum_imp_client"].astype("Int64")
grouped_df["sum_fixations"] = grouped_df["sum_fixations"].astype("Int64")
grouped_df["sum_fixations_client"] = grouped_df["sum_fixations_client"].astype("Int64")
grouped_df["sum_inview"] = grouped_df["sum_inview"].astype("Int64")
grouped_df["sum_inview_client"] = grouped_df["sum_inview_client"].astype("Int64")


grouped_df

,domain,pid,sum_imp,sum_imp_client,sum_fixations,sum_fixations_client,sum_inview,sum_inview_client,avg_weight
0,9gag.com,15582485,4,6586,0,0,4,6586,0.5
1,aftonbladet.se,14745815,4,138,4,69,4,138,0.5
2,aftonbladet.se,14746054,53,13,4,0,22,5,0.7
3,aftonbladet.se,14746055,48,2,7,0,23,0,0.7
4,aftonbladet.se,14746059,296,28,25,2,217,20,1.0
...,...,...,...,...,...,...,...,...,...
266,vibilagare.se,24928806,1,25,0,0,1,25,0.5
267,vibilagare.se,24928807,13,91,1,3,8,56,0.5
268,vistoenlasredes.com,7189038,1,2016,0,0,0,0,0.5
269,warcraftlogs.com,21997200,8,3787,0,0,0,0,0.5


In [49]:
result_df = grouped_df.groupby(["domain"]).apply(
    lambda x: pd.Series(
        {
            "weight": np.average(x["avg_weight"]),
            "impressions": x["sum_imp"].sum(),
            "impressions_client": x["sum_imp_client"].sum(),
            "fixations": x["sum_fixations_client"].sum(),
            "in_views": x["sum_inview_client"].sum(),
            #"fixation_inview_ratio": np.average("avg_fixation_inview_ratio"),
            "Placement IDs": list(x["pid"]),
        }
    )
)
result_df["fixation_ratio"] = round(result_df["fixations"] / result_df["impressions_client"] * 100, 2)
result_df["fixation/inview_ratio"] = round(result_df["fixations"] / result_df["in_views"] * 100, 2)
result_df["fixation/inview_ratio"] = result_df["fixation/inview_ratio"].fillna(0)
result_df["fixation/inview_ratio"] = result_df["fixation/inview_ratio"].replace([np.inf, -np.inf], 0)
result_df["sample_size"] = result_df["impressions"].apply(lambda x: get_sample_size(x))
result_df = result_df.reset_index()
result_df = result_df.sort_values(by=["impressions_client"], ascending=False)
final_df = result_df[["domain", "impressions_client", "fixations", "in_views", "fixation_ratio", "fixation/inview_ratio", "sample_size", "Placement IDs"]]
final_df = final_df.rename(columns={"impressions_client": "impressions"})
final_df.to_excel("../final_reports/tele2_groupm_campaign_v3.xlsx", index=False)
final_df

,domain,impressions,fixations,in_views,fixation_ratio,fixation/inview_ratio,sample_size,Placement IDs
39,ifokus.se,875432,192419,162830,21.98,118.17,medium,[6931042]
71,psu.com,264191,5363,76747,2.03,6.99,high,[28082693]
53,macworld.com,203056,0,58987,0.00,0.00,high,[18263083]
13,cyclingnews.com,114965,5345,20233,4.65,26.42,high,[25659495]
47,lag-avtal.se,100667,0,33552,0.00,0.00,low,[13255186]
...,...,...,...,...,...,...,...,...
59,moviezine.se,2,0,0,0.00,0.00,low,[18490171]
56,mobil.se,2,0,0,0.00,0.00,low,[18160151]
20,ekstrabladet.dk/musik,1,0,0,0.00,0.00,low,[24409812]
34,helpdeskgeek.com,1,0,0,0.00,0.00,low,[24411573]
